In [4]:
import os
from openai import OpenAI

client = OpenAI(api_key = os.getenv("OPENAI_TEST_KEY"))


In [5]:
os.listdir()

['data', 'test_openAI_API.ipynb']

In [9]:
# Read the pdf with the CV information
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("./data/CV_AntonioOchotorena_092024.pdf")
pages = loader.load()
pages

[Document(metadata={'source': './data/CV_AntonioOchotorena_092024.pdf', 'page': 0}, page_content='ANTONIO OCHOTORENA  LAYNEZ  \n+34 636 426 538  • antonioochotorena@gmail.com  • LinkedIn  • GitHub  \n \nSummary   \n \nActively seeking a full -time opportunity in data and AI in Amsterdam.  My background in consulting \nand data science has equipped me with the skills to excel in a position where effective \ncommunication and technical expertise are essential . What sets me apart is my strong drive to learn, \nproactive approach to solving complex problems, and commitment to fostering a collaborative team \nenvironment where we support and help each other to achieve shared goals.  \n \nExperience   \n \nInfosys  Instep – Global Internship Program  Bangalore, India  \nAI Software Engineer  06/2024 – Now  \n \n• Developed a proprietary information retrieval module utilizing Large Language Models \n(LLMs) and OCR technologies to extract healthcare forms, achieving substantial cost savings. 

In [ ]:
# Split the text 

In [ ]:
# Store it into a ChromaDB database


In [12]:
def get_completion(prompt, client):
    
    completion = client.chat.completions.create(
                    model="gpt-3.5-turbo-0125",
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": prompt}
                    ],
                    max_tokens=1000,
                    temperature=0,
                )
    return completion

In [13]:
text = pages[0].page_content

prompt = f"""
Please follow the following format:
<role 1> <reasons based on experience> <areas of improvement towards role 1>
<role 2> <reasons based on experience> <areas of improvement towards role 2>
<role 3> <reasons based on experience> <areas of improvement towards role 3>

Do not do more than 3 roles
```{text}```
"""
result = get_completion(prompt, client)


In [14]:
print(result.choices[0].message.content)

Data Engineer at Infosys Instep in Amsterdam
Antonio has a strong background in data engineering, as evidenced by his successful DWH platform migration for Beam Suntory Spain and the development of a Support Decision System using Deep Learning at Ramón Y Cajal Hospital. He has experience in leading projects, implementing reporting tools, and designing NLP pipelines for biomedical document classification. Antonio's technical skills in Python, R, SQL, and various data tools make him well-suited for a Data Engineer role.

Areas of improvement:
- Antonio could focus on enhancing his knowledge of cloud platforms like AWS and Azure to stay updated with the latest technologies in data engineering.
- Developing expertise in big data technologies like Spark and Hadoop would further strengthen his profile for data engineering roles.

AI Software Engineer in Amsterdam
Antonio's experience in developing an information retrieval module using Large Language Models and OCR technologies at Infosys Ins